In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_reservation = pl.read_parquet(datadir / "reservation.parquet")
pl_hotel_sales = pl.read_parquet(datadir / "hotel_sales.parquet")

# 9章 整形
## 9-1 縦持ちデータから横持ちデータへの変換
### Q: 横持ち変換


#### Awesome

In [ ]:
pl_reservation.pivot(values="reservation_id", index="hotel_id", columns="people_num",
                     aggregate_function="len", sort_columns=True)

## 9-2 横持ちから縦持ちへの変換
### Q: 縦持ち変換


#### Awesome

In [ ]:
pl_hotel_sales.melt(id_vars="month", variable_name="hotel_type", value_name="sales")

## 9-3 スパースマトリックスへの変換
### Q: 顧客×宿泊人数ごとの予約数のスパースマトリックス作成


#### Awesome

In [ ]:
from scipy.sparse import csr_matrix

# （1） 顧客・宿泊人数ごとの予約数を集計した縦持ちデータを作成
cnt_data = pl_reservation.group_by(["customer_id", "people_num"]).agg(pl.len())

# （2） スパースマトリックスの行／列にする列をカテゴリ型に変換
customer_id = cnt_data["customer_id"].cast(pl.Utf8).cast(pl.Categorical)
people_num_id = cnt_data["people_num"].cast(pl.Utf8).cast(pl.Categorical)

# （3） スパースマトリックス変換
csr_matrix((
    cnt_data["len"],
    (customer_id.to_physical().to_numpy(), people_num_id.to_physical().to_numpy())
))